<h1>Zindi Air Pollution </h1> 

In [243]:
import pandas as pd
import os
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor as rfr
os.chdir('c:\\Users\\Pascal\\Downloads')
Dataset = pd.read_csv('Train.csv')

In [248]:
#adding Quaters to dataset
Timedata = Dataset[['id', 'country', 'city', 'date', 'hour']]
Timedata['Quater'] = pd.to_datetime(Timedata['date']).dt.to_period('Q')
Q = []
for i in Timedata['Quater']:
     Q.append(str(i)[-1:])
Dataset['Quaters'] = Q
Timedata.drop('Quater', axis = 1, inplace= True)

C:\Users\Pascal\AppData\Local\Temp/ipykernel_11928/2419470941.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Timedata['Quater'] = pd.to_datetime(Timedata['date']).dt.to_period('Q')
c:\Users\Pascal\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
#Correlation checks
cors = abs(Dataset.corr()['pm2_5']).sort_values(ascending=False)[:20]

In [251]:
target = 'pm2_5'
non_preds = ['id','site_id', 'site_latitude', 'site_longitude', 'city',  'date', 'hour', target]

In [252]:
##fill null values with mean from groupby quater

features = Dataset.drop(non_preds, axis= 1)
grouped = features.groupby(['country', 'Quaters']).mean()
for i in features.columns:
    fields = features[i]
    dex = 0
    for u in fields:
        period = features['Quaters'][dex]
        place = features['country'][dex]
        #print(place)
        if pd.isna(u):
            features.loc[dex,i] = grouped.loc[place,period][i]
            #print(u)
        dex += 1

            


In [255]:
non_preds.append('country')
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(Dataset.drop(non_preds, axis=1).bfill().ffill(), Dataset[target], test_size= 0.4)
model = rfr(n_estimators = 50, max_depth = 30, random_state = 2, bootstrap= True, max_samples= 0.3).fit(X=X_train, y=y_train)
y_pred = model.predict(X_test)
#from sklearn.metrics import report
#y_pred
model.score(X_test, y_test)

0.37729842557513593